In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Matrix for computing forward model

We want to create a function that gets the depths of the geophones and the proposed layer structure of the subsurface to compute the **G** matrix defined as follows:

$$ \bf{d} = \bf{G} \bf{m} $$

where $\bf{d}$ is the travel times vector and $\bf{m}$ is the slowness vector of the layers.

In [2]:
def get_G_matrix(geophones_depth, layer_thickness, n_layers):
    """
    Compute matrix G for VSP forward model
    
    .. math ::
        d = Gm
    
    Where `d` is the travel times vector and `m` is the slowness vector,
    whose elements correspond to the slowness of each layer of the model.
    
    Parameter
    ---------
    geophones_depth : array
        Array of depths of the geophones
    layer_thickness : float
        Thickness of the layers, assuming each layer has the same
        thickness.
    n_layers : int
        Total number of layers of same thickness.
        
    Returns
    -------
    G : numpy.matrix
        Matrix G that performs the VSP forward modelling
    """
    # Check if the layer structure is well defined
    if n_layers * layer_thickness < max(geophones_depth):
        raise ValueError(
            "Layer structure must be as deep as the deepest geophone"
        )
    # Initialize G matrix full of zeros
    n_geophones = len(geophones_depth)
    G = np.matrix(np.zeros((n_geophones, n_layers)))
    # Compute elements of G
    for i, depth in enumerate(geophones_depth):
        # Get the number of the layer where the geophone is located
        layer_of_geophone = int(depth // layer_thickness)
        G[i, :layer_of_geophone] = layer_thickness
        G[i, layer_of_geophone] = depth % layer_thickness
    return G

# Read data from `data_vsp.asc`

In [3]:
geophones_depth, travel_times = np.loadtxt("data_vsp.asc", unpack=True)

In [4]:
n_geophones = len(geophones_depth)

# Define inverse problem

We must propose a subsurface discretized model. We will asume N layers of equal thickess.

In [54]:
n_layers = 100
max_depth = 1
layer_thickness = max_depth / n_layers

We will impose that our model should be near a reference model of constant slowness of 1/3 s/km.

In [55]:
reference_model = 1/3 * np.ones(n_layers)

The cost function we want to minimize can be written as follows:

$$ J = || d - Gm ||^2 + \mu^2 || D (m - m_\text{ref}) ||^2 $$

where $||\cdot||^2$ notes the $L_2$ norm, $m_\text{ref}$ is the reference model, $D$ is the identity matrix and $\mu$ is the regularization paramter (a.k.a. trade-off parameter).

The model that minimizes this cost function can be computed as follows:

$$ m_\text{est} = \left[ G^T W_e G + \mu^2 D^T D \right]^{-1} \left[ G^T W_e d + \mu^2 D^T D m_\text{ref} \right]$$

where $W_e$ is the diagonal matrix of data variances.

In [56]:
sigma = 0.0018
W_e = sigma * np.matrix(np.identity(n_geophones))

D = np.matrix(np.identity(n_layers))

Get G matrix

In [57]:
G = get_G_matrix(geophones_depth, layer_thickness, n_layers)

In [58]:
mu = 1e-5
A = np.dot(G.T , np.dot(W_e, G)) + mu ** 2 * np.dot(D.T, D)
B = (
    np.dot(G.T, np.dot(W_e, travel_times[:, np.newaxis]))
    + mu ** 2 * np.dot(D.T , np.dot(D, reference_model[:, np.newaxis]))
)

In [59]:
np.linalg.solve(A, B)

matrix([[ 0.34335817],
        [ 0.34335817],
        [ 0.34335817],
        [ 0.34335817],
        [ 0.34335817],
        [ 0.34335817],
        [ 0.34335817],
        [ 0.34335817],
        [ 0.34335817],
        [ 0.34335817],
        [ 0.15331297],
        [ 0.42871026],
        [ 0.38994919],
        [ 0.21321963],
        [ 0.5081898 ],
        [ 0.33541299],
        [ 0.32826986],
        [ 0.09135706],
        [ 0.54121218],
        [ 0.11263955],
        [ 0.54128613],
        [ 0.20645179],
        [ 0.29063712],
        [ 0.30302253],
        [ 0.56640794],
        [ 0.32244159],
        [ 0.23177612],
        [ 0.50355302],
        [ 0.20482727],
        [ 0.31677224],
        [ 0.20848056],
        [ 0.48988344],
        [ 0.32897097],
        [ 0.3453105 ],
        [ 0.27780947],
        [ 0.2314935 ],
        [ 0.25516713],
        [ 0.59364119],
        [ 0.12370981],
        [ 0.44982723],
        [ 0.38871252],
        [-0.08209182],
        [ 0.69532569],
        [ 0